In [161]:
import pandas as pd

import numpy as np

import pickle

import csv

from scipy.stats import pearsonr

from scipy.stats import f_oneway

In [2]:
# Abrindo o objeto dataframe
with open(r'df-tokenizado.pickle', 'rb') as pickledfile:
    df = pickle.load(pickledfile)

In [3]:
df.head()

,Assunto,Texto original,Texto semi-tratado,Texto tratado,Tokenizadas
0,Direito Tributário,\nEmenta: * IMPOSTO - Propriedade Predial e Te...,* IMPOSTO - Propriedade Predial e Territorial ...,* IMPOSTO - Propriedade Predial e Territorial ...,"[101, 115, 290, 11881, 6072, 15349, 118, 1072,..."
1,Direito Tributário,\nEmenta: ACÓRDÃO\nMANDADO DE SEGURANÇA - IPTU...,ACÓRDÃO MANDADO DE SEGURANÇA - IPTU - Progress...,ACÓRDÃO MANDADO DE SEGURANÇA - IPTU - Progress...,"[101, 11247, 22369, 22322, 22319, 16484, 18471..."
2,Direito Tributário,\nEmenta: da função social do imóvel para se t...,da função social do imóvel para se tornar um t...,da função social do imóvel para se tornar um t...,"[101, 180, 2403, 1979, 171, 16068, 221, 176, 2..."
3,Direito Tributário,\nEmenta: ACÓRDÃO\nAGRAVO DE INSTRUMENTO - INT...,ACÓRDÃO AGRAVO DE INSTRUMENTO - INTERPOSIÇÃO C...,ACÓRDÃO AGRAVO DE INSTRUMENTO - INTERPOSIÇÃO C...,"[101, 11247, 22369, 22322, 22319, 16484, 177, ..."
4,Direito Tributário,\n\n\t\t\t\t\t\t\t\t\t\t\t\t\tEmenta:\n\t\t\t\...,IMPOSTO. Predial e Territorial Urbano. Municíp...,IMPOSTO. Predial e Territorial Urbano. Municíp...,"[101, 290, 11881, 6072, 15349, 119, 4407, 679,..."


In [18]:
unicos = np.unique(df['Assunto'])

In [22]:
dict_minidx = {}

In [23]:
for area in unicos:
    dict_minidx.update(
        {
            area: df.loc[df['Assunto'] == area].index.min()
        }
    )

In [157]:
df.loc[:, 'idx_categoria'] = [i - dict_minidx[df['Assunto'][i]] + 1 for i in range(len(df))]

In [158]:
df.loc[:, 'idx_categoria2'] = [df['idx_categoria'][i]//2500 + 1 for i in range(len(df))]

In [34]:
# Abrindo o objeto tfidf
with open(r'tfidf.pickle', 'rb') as pickledfile:
    tfidf = pickle.load(pickledfile)

In [36]:
# Abrindo o objeto tfidf features
with open(r'tfidf_features.pickle', 'rb') as pickledfile:
    tfidf_features = pickle.load(pickledfile)

In [56]:
# validando se os índices estão corretos
i = 70000
j = 4
tfidf[i][np.where(tfidf_features == str(df['Tokenizadas'][i][j]))[0][0]]

0.010163525101390293

### Análise de Pearson

In [192]:
pr = []

for i in range(tfidf.shape[1]):
    row = pearsonr(df['idx_categoria'], tfidf[:, i])
    pr.append([i, row.statistic, row.pvalue])

In [197]:
def filter_pr(x):
    if x[2]<=0.05:
        return True
    else:
        return False

filtered = [row for row in filter(filter_pr, pr)]

def sort_pr(x):
    return abs(x[1])

sorted(filtered, key=sort_pr, reverse=True)

[[3241, 0.23813843341844262, 0.0],
 [0, 0.15433910802396902, 0.0],
 [9451, -0.09769563891861208, 9.476270683939177e-160],
 [11050, -0.09685750500558034, 4.828541254294996e-157],
 [14285, -0.09514530916999916, 1.3822052997628613e-151],
 [1579, -0.08739284289974436, 4.0986195721832095e-128],
 [10203, -0.08499707731949803, 2.870762881806002e-121],
 [335, -0.08130204401936243, 4.352847971144992e-111],
 [10574, 0.08038392107347771, 1.2519637207280079e-108],
 [11381, -0.07961021133189643, 1.4053450974651257e-106],
 [9901, -0.07953198291144908, 2.2592317732511316e-106],
 [11567, -0.07547725186023482, 5.765471400068818e-96],
 [1469, -0.07540284895818959, 8.84449477235656e-96],
 [3230, -0.07524117120988895, 2.2380196012542678e-95],
 [10197, 0.0736922939440095, 1.473322561957678e-91],
 [6646, 0.07346113139782909, 5.386752490751249e-91],
 [5706, 0.0731233893616557, 3.5542367355888625e-90],
 [6782, -0.07210517186917856, 9.957066207228228e-88],
 [11366, 0.07186831398377727, 3.6517011556915503e-87],

### Análise de Pearson 2

In [186]:
pr2 = []

for i in range(tfidf.shape[1]):
    row = pearsonr(df['idx_categoria2'], tfidf[:, i])
    pr2.append([i, row.statistic, row.pvalue])

In [191]:
def filter_pr2(x):
    if x[2]<=0.05 and abs(x[1])>0.1:
        return True
    else:
        return False

filtered_pr2 = [row for row in filter(filter_pr2, pr2)]

filtered_pr2

[[0, 0.15332632556590967, 0.0], [3241, 0.24045676991292464, 0.0]]

### Análise ANOVA

In [179]:
max(df['idx_categoria2'])

7

In [181]:
anv1 = []

for i in range(tfidf.shape[1]):

    sp1 = tfidf[df.loc[df['idx_categoria2'] == 1].index.values, i]
    sp2 = tfidf[df.loc[df['idx_categoria2'] == 2].index.values, i]
    sp3 = tfidf[df.loc[df['idx_categoria2'] == 3].index.values, i]
    sp4 = tfidf[df.loc[df['idx_categoria2'] == 4].index.values, i]
    sp5 = tfidf[df.loc[df['idx_categoria2'] == 5].index.values, i]


    row1 = f_oneway(sp1,sp2,sp3,sp4,sp5)
    anv1.append([i, row.statistic, row.pvalue, 
                np.average(sp1), np.average(sp2), np.average(sp3), np.average(sp4), np.average(sp5)])

c:\Users\Lucas Alponti\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:4167: ConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(stats.ConstantInputWarning(msg))


In [184]:
def filter_anv1(x):
    if x[2]<=0.01:
        return True
    else:
        return False

filtered_anv1 = [row for row in filter(filter_anv1, anv1)]

filtered_anv1

[[0,
  14.635827294546894,
  5.930770821099298e-12,
  0.0037440676702296858,
  0.0037865538401125813,
  0.010111237339165453,
  0.015361095234624922,
  0.018760868394107292],
 [1,
  14.635827294546894,
  5.930770821099298e-12,
  0.002481488497603619,
  0.0019514483409457705,
  0.001994455619376787,
  0.0018814448368495351,
  0.002235514693653174],
 [2,
  14.635827294546894,
  5.930770821099298e-12,
  1.1257036224810375e-05,
  4.7208130146781055e-06,
  2.283046098577378e-05,
  2.7675537090761536e-05,
  0.0],
 [3,
  14.635827294546894,
  5.930770821099298e-12,
  0.0,
  8.625579814642379e-06,
  0.0,
  0.0,
  0.0],
 [4,
  14.635827294546894,
  5.930770821099298e-12,
  3.778834343465793e-06,
  0.0,
  0.0,
  0.0,
  0.0],
 [5,
  14.635827294546894,
  5.930770821099298e-12,
  1.178763184092758e-05,
  0.0,
  0.0,
  0.0,
  1.3566611077885125e-05],
 [6,
  14.635827294546894,
  5.930770821099298e-12,
  0.0001364778298759959,
  3.996309090046409e-05,
  3.9945550304565044e-05,
  0.000100678719042117

In [185]:
len(filtered_anv1)

17200

### Análise ANOVA 2

In [174]:
anv = []

for i in range(tfidf.shape[1]):

    cat1 = tfidf[df.loc[df['Assunto'] == unicos[0]].index.values, i]
    cat2 = tfidf[df.loc[df['Assunto'] == unicos[1]].index.values, i]
    cat3 = tfidf[df.loc[df['Assunto'] == unicos[2]].index.values, i]
    cat4 = tfidf[df.loc[df['Assunto'] == unicos[3]].index.values, i]
    cat5 = tfidf[df.loc[df['Assunto'] == unicos[4]].index.values, i]
    row = f_oneway(cat1,cat2,cat3,cat4,cat5)
    anv.append([i, row.statistic, row.pvalue, 
                np.average(cat1), np.average(cat2), np.average(cat3), np.average(cat4), np.average(cat5)])

In [178]:
def filter_anv(x):
    if x[2]<=0.05:
        return True
    else:
        return False

filtered_anv = [row for row in filter(filter_anv, anv)]

filtered_anv

[[5,
  0.715051676552063,
  0.581488096202604,
  0.0,
  1.105204890990381e-05,
  0.0,
  9.242953238304996e-06,
  0.0],
 [28,
  0.7952756484868457,
  0.5279926255588545,
  4.5470767459146345e-06,
  0.0,
  1.6935393430642146e-05,
  0.0,
  0.0],
 [33,
  0.5576992783981727,
  0.6933957176272402,
  3.893554029623041e-06,
  5.382517364692966e-06,
  9.551841171804612e-06,
  0.0,
  0.0],
 [80,
  0.827323839825022,
  0.5074653700149174,
  1.5156070707849334e-05,
  2.9431600361652516e-05,
  0.0,
  0.0,
  0.0],
 [90,
  0.8046348677459358,
  0.5219458917541175,
  4.5941045348952904e-06,
  0.0,
  0.0,
  1.0749053338907884e-05,
  0.0],
 [93,
  0.7194414945460823,
  0.5784877433805591,
  0.0,
  0.0,
  1.0764493166879649e-05,
  0.0,
  1.165821961230388e-05],
 [97,
  0.5959383316324397,
  0.6655663689825473,
  0.0,
  5.823320554007716e-06,
  2.0897980898986215e-05,
  0.0,
  1.0721704492253062e-05],
 [145,
  0.5738965682289341,
  0.6815735766479417,
  5.230854022402987e-06,
  7.63542847199225e-06,
  0.0

In [167]:
len(unicos)

5